Steps to run this Code:
1. Run cell 1 to import required Libraries.
2. Run cell 2 for Mounting Google drive where Dataset is located.
3. Run Cell 3 to do data preprocessing and covert images.
4. Run cell 4 for defining Discriminator Model.
5. To get final results: don't run cell 5: this cell including the very first design which got improved on the following cell 6.
6. Run Cell 6 for defining Generator Model
7. Run the rest cell to build and compile GAN model.
8. Make sure to insert correct Path according to your environment: all included Pathes below is according to my Colab and Google drive. If you follow my step it will work with you just provide the correct path for dataset in your Googlke drive and then my code will create the other needed folders and copy data to your colab.
9. You can download Dataset from Kaggle: https://www.kaggle.com/datasets/soumikrakshit/lol-dataset

# Fetching Dataset from Google Drive

In [37]:
import os
import time
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import randint
from tensorflow.keras import Input
from numpy import load, zeros, ones
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, Activation
from tensorflow.keras.layers import Concatenate, Dropout, BatchNormalization, LeakyReLU

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Add, Concatenate
from tensorflow.keras.models import Model
import numpy as np

In [39]:
#Importing Data
from google.colab import drive
drive.mount('/content/drive')

import shutil


#shutil.rmtree("/content/gans_new2/") # use this line only in case you already copied data and you are trying to recopy it again for any reason.
shutil.copytree("/content/drive/MyDrive/LOLdataset/", "/content/gans_new2/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/gans_new2/'

#Importing Libraries

In [40]:
#Data Preprocessing

################################ Imports ####################################
from os import listdir
from keras.preprocessing.image import img_to_array,load_img
from numpy import asarray, savez_compressed


################################ Load Dataset ####################################

def load_images(path, size=(256,256)):
    data_list = list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # store
        data_list.append(pixels)

    return asarray(data_list)


################################ Main Function ####################################
def main():

   # load images

   path = "/content/gans_new2/our485/"

   high = load_images(path + 'high/')   #directory "path/high" should be present
   low = load_images(path + 'low/')             #directory "path/low" should be present

   savez_compressed('dataset.npz',high,low)

if __name__ == '__main__':
    main()


# Creating Functions

## Defining Discriminator

In [41]:
def define_discriminator(image_shape):
	init = RandomNormal(stddev=0.02)

	in_src_image = Input(shape=image_shape)
	in_target_image = Input(shape=image_shape)

	merged = Concatenate()([in_src_image, in_target_image])

	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)

	model = Model([in_src_image, in_target_image], patch_out)

	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

## Defining Generator

For Final Result Don't Run below cell, just skip it and run the one after.

In [21]:
#For Final Results don't run this cell: this cell contains the very first model design

def define_generator(image_shape = (256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)

    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = BatchNormalization()(g, training=True)
    g3 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g3)
    g = BatchNormalization()(g, training=True)
    g2 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g2)
    g = BatchNormalization()(g, training=True)
    g1 = LeakyReLU(alpha=0.2)(g)

    for _ in range(6):
        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g1)
        g = BatchNormalization()(g, training=True)
        g = LeakyReLU(alpha=0.2)(g)

        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g)
        g = BatchNormalization()(g, training=True)

        g1 = Concatenate()([g, g1])

    g = UpSampling2D((2, 2))(g1)
    g = Conv2D(128, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g2])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = UpSampling2D((2, 2))(g)
    g = Conv2D(64, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g3])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = BatchNormalization()(g, training=True)
    out_image = Activation('tanh')(g)

    model = Model(in_image, out_image)
    return model

In [51]:
#Final Generator Model: Use this Cell for Final results

# Define the generator model for low-light image enhancement
def define_generator(image_shape = (256, 256, 3)):
    #inputs = Input(shape=input_shape)
    in_image = Input(shape=image_shape)

    # Preprocessing layer
    attention_map = tf.subtract(1.0, tf.image.rgb_to_grayscale(in_image))
    enhanced_inputs = tf.add(in_image, attention_map)

    # Initial convolution
    x = Conv2D(32, (3, 3), padding='same')(enhanced_inputs)
    x = LeakyReLU()(x)

    # Module 1
    residual1 = x
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    residual1 = Conv2D(64, (1, 1), padding='same')(residual1)
    x = Add()([x, residual1])
    x = LeakyReLU()(x)
    downsampled1 = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)

    # Module 2
    residual2 = downsampled1
    x = Conv2D(64, (3, 3), padding='same')(downsampled1)
    x = LeakyReLU()(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    residual2 = Conv2D(64, (1, 1), padding='same')(residual2)
    x = Add()([x, residual2])
    x = LeakyReLU()(x)
    downsampled2 = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)

    # Upsampling and concatenation
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(downsampled2)
    x = Concatenate()([x, downsampled1])

    # Residual module
    residual3 = x
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    residual3 = Conv2D(64, (1, 1), padding='same')(residual3)
    x = Add()([x, residual3])
    x = LeakyReLU()(x)

    # Upsampling and concatenation
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = Concatenate()([x, in_image])

    # Final convolution
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    out_image = Conv2D(3, (3, 3), padding='same', activation='tanh')(x)

    #model = Model(inputs, outputs)
    model = Model(in_image, out_image)
    return model



'\n# Example usage\ninput_shape = (256, 256, 3)  # Input image shape\ngenerator = build_generator(input_shape)\ngenerator.summary()\n\n# Build the generator model\ninput_shape = (256, 256, 3)  # Input image shape\ngenerator = build_generator(input_shape)\n\n# Print the summary of the generator model\ngenerator.summary()\n'

## Initializing GAN training

In [52]:
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

## Loading Real Samples

In [53]:
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X2, X1]

## Generating Real Fake Samples

In [54]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

## Generating Fake Samples

In [55]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

## Summarizing Training and Saving Model

In [56]:
def summarize_performance(step, g_model, d_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    plt.figure(figsize=(14, 14))
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.title('Low-Light')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.title('Generated')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.title('High-Light')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = step_output + 'plot_%06d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = model_output + 'gen_model_%06d.h5' % (step+1)
    g_model.save(filename2)
    # save the discriminator model
    filename3 = model_output + 'disc_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('[.] Saved Step : %s' % (filename1))
    print('[.] Saved Model: %s' % (filename2))
    print('[.] Saved Model: %s' % (filename3))

## Training Function

In [57]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=12):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print("[!] Number of steps {}".format(n_steps))
    print("[!] Saves model/step output at every {}".format(bat_per_epo * 1))
    # manually enumerate epochs
    for i in range(n_steps):
        start = time.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        time_taken = time.time() - start
        print(
            '[*] %06d, d1[%.3f] d2[%.3f] g[%06.3f] ---> time[%.2f], time_left[%.08s]'
                %
            (i+1, d_loss1, d_loss2, g_loss, time_taken, str(datetime.timedelta(seconds=((time_taken) * (n_steps - (i + 1))))).split('.')[0].zfill(8))
        )
        # summarize model performance
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, d_model, dataset)

# Main Function

## Loading Dataset

In [58]:
dataset = load_real_samples('/content/dataset.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
print(image_shape)

Loaded (485, 256, 256, 3) (485, 256, 256, 3)
(256, 256, 3)


## Defining Models

In [59]:
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


## Creating model Directory and Calling Train Function

In [60]:
#dir = '<enter path to save model on colab>'
dir = '/content/'
fileName = 'Enhancement Model'
step_output = dir + fileName + "/Step Output/"
model_output = dir + fileName + "/Model Output/"
if fileName not in os.listdir(dir):
    os.mkdir(dir + fileName)
    os.mkdir(step_output)
    os.mkdir(model_output)

train(d_model, g_model, gan_model, dataset, n_epochs=10, n_batch=12)

[!] Number of steps 400
[!] Saves model/step output at every 40
1/1 [==============================] - 0s 212ms/step
[*] 000001, d1[0.286] d2[2.601] g[41.387] ---> time[10.16], time_left[01:07:33]
1/1 [==============================] - 0s 23ms/step
[*] 000002, d1[1.163] d2[1.999] g[42.945] ---> time[0.78], time_left[00:05:09]
1/1 [==============================] - 0s 22ms/step
[*] 000003, d1[0.286] d2[0.731] g[40.821] ---> time[0.72], time_left[00:04:44]
1/1 [==============================] - 0s 24ms/step
[*] 000004, d1[0.333] d2[0.535] g[33.866] ---> time[0.72], time_left[00:04:46]
1/1 [==============================] - 0s 30ms/step
[*] 000005, d1[0.496] d2[0.394] g[35.067] ---> time[0.74], time_left[00:04:53]
1/1 [==============================] - 0s 24ms/step
[*] 000006, d1[0.448] d2[0.359] g[33.702] ---> time[0.77], time_left[00:05:04]
1/1 [==============================] - 0s 27ms/step
[*] 000007, d1[0.373] d2[0.464] g[33.328] ---> time[0.92], time_left[00:06:02]
1/1 [============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000040.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000040.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000040.h5
1/1 [==============================] - 0s 35ms/step
[*] 000041, d1[0.005] d2[0.006] g[37.229] ---> time[0.93], time_left[00:05:34]
1/1 [==============================] - 0s 26ms/step
[*] 000042, d1[0.004] d2[0.009] g[40.375] ---> time[0.85], time_left[00:05:05]
1/1 [==============================] - 0s 24ms/step
[*] 000043, d1[0.005] d2[0.006] g[40.691] ---> time[0.77], time_left[00:04:34]
1/1 [==============================] - 0s 24ms/step
[*] 000044, d1[0.003] d2[0.006] g[42.425] ---> time[0.81], time_left[00:04:48]
1/1 [==============================] - 0s 24ms/step
[*] 000045, d1[0.003] d2[0.006] g[39.894] ---> time[0.74], time_left[00:04:20]
1/1 [==============================] - 0s 23ms/step
[*] 000046, d1[0.003] d2[0.006] g[41.048] ---> time[0.80], time_le

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000080.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000080.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000080.h5
1/1 [==============================] - 0s 25ms/step
[*] 000081, d1[0.004] d2[0.010] g[40.535] ---> time[0.84], time_left[00:04:26]
1/1 [==============================] - 0s 26ms/step
[*] 000082, d1[0.005] d2[0.007] g[44.974] ---> time[0.73], time_left[00:03:51]
1/1 [==============================] - 0s 24ms/step
[*] 000083, d1[0.004] d2[0.007] g[40.720] ---> time[0.85], time_left[00:04:30]
1/1 [==============================] - 0s 33ms/step
[*] 000084, d1[0.006] d2[0.010] g[35.986] ---> time[1.16], time_left[00:06:05]
1/1 [==============================] - 0s 71ms/step
[*] 000085, d1[0.003] d2[0.009] g[41.568] ---> time[1.15], time_left[00:06:03]
1/1 [==============================] - 0s 38ms/step
[*] 000086, d1[0.004] d2[0.006] g[41.091] ---> time[0.79], time_le

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000120.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000120.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000120.h5
1/1 [==============================] - 0s 25ms/step
[*] 000121, d1[0.019] d2[0.008] g[35.437] ---> time[0.81], time_left[00:03:47]
1/1 [==============================] - 0s 23ms/step
[*] 000122, d1[0.007] d2[0.004] g[43.523] ---> time[0.72], time_left[00:03:18]
1/1 [==============================] - 0s 23ms/step
[*] 000123, d1[0.003] d2[0.004] g[43.890] ---> time[0.80], time_left[00:03:42]
1/1 [==============================] - 0s 23ms/step
[*] 000124, d1[0.002] d2[0.004] g[46.353] ---> time[0.77], time_left[00:03:32]
1/1 [==============================] - 0s 26ms/step
[*] 000125, d1[0.003] d2[0.004] g[37.524] ---> time[0.73], time_left[00:03:21]
1/1 [==============================] - 0s 31ms/step
[*] 000126, d1[0.001] d2[0.004] g[44.085] ---> time[0.77], time_le

...

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000240.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000240.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000240.h5
1/1 [==============================] - 0s 23ms/step
[*] 000241, d1[0.001] d2[0.005] g[38.686] ---> time[0.84], time_left[00:02:13]
1/1 [==============================] - 0s 23ms/step
[*] 000242, d1[0.001] d2[0.006] g[40.473] ---> time[0.72], time_left[00:01:54]
1/1 [==============================] - 0s 23ms/step
[*] 000243, d1[0.001] d2[0.005] g[46.194] ---> time[0.88], time_left[00:02:18]
1/1 [==============================] - 0s 32ms/step
[*] 000244, d1[0.001] d2[0.006] g[35.393] ---> time[1.17], time_left[00:03:02]
1/1 [==============================] - 0s 30ms/step
[*] 000245, d1[0.001] d2[0.006] g[38.236] ---> time[1.06], time_left[00:02:44]
1/1 [==============================] - 0s 41ms/step
[*] 000246, d1[0.001] d2[0.005] g[37.330] ---> time[0.92], time_le

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000280.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000280.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000280.h5
1/1 [==============================] - 0s 25ms/step
[*] 000281, d1[0.003] d2[0.007] g[32.934] ---> time[0.82], time_left[00:01:37]
1/1 [==============================] - 0s 26ms/step
[*] 000282, d1[0.002] d2[0.006] g[41.928] ---> time[0.81], time_left[00:01:35]
1/1 [==============================] - 0s 27ms/step
[*] 000283, d1[0.002] d2[0.009] g[37.011] ---> time[0.81], time_left[00:01:34]
1/1 [==============================] - 0s 23ms/step
[*] 000284, d1[0.001] d2[0.006] g[35.237] ---> time[0.72], time_left[00:01:23]
1/1 [==============================] - 0s 34ms/step
[*] 000285, d1[0.001] d2[0.008] g[32.975] ---> time[1.24], time_left[00:02:22]
1/1 [==============================] - 0s 33ms/step
[*] 000286, d1[0.002] d2[0.007] g[33.219] ---> time[0.87], time_le

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000320.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000320.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000320.h5
1/1 [==============================] - 0s 23ms/step
[*] 000321, d1[0.046] d2[0.121] g[92.236] ---> time[0.83], time_left[00:01:05]
1/1 [==============================] - 0s 24ms/step
[*] 000322, d1[0.043] d2[0.063] g[93.235] ---> time[0.83], time_left[00:01:04]
1/1 [==============================] - 0s 26ms/step
[*] 000323, d1[0.035] d2[0.052] g[97.279] ---> time[0.71], time_left[00:00:54]
1/1 [==============================] - 0s 25ms/step
[*] 000324, d1[0.033] d2[0.090] g[97.473] ---> time[0.72], time_left[00:00:54]
1/1 [==============================] - 0s 25ms/step
[*] 000325, d1[0.034] d2[0.042] g[93.849] ---> time[0.74], time_left[00:00:55]
1/1 [==============================] - 0s 30ms/step
[*] 000326, d1[0.030] d2[0.038] g[100.488] ---> time[0.82], time_l

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000360.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000360.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000360.h5
1/1 [==============================] - 0s 36ms/step
[*] 000361, d1[0.007] d2[0.015] g[92.040] ---> time[1.21], time_left[00:00:47]
1/1 [==============================] - 0s 34ms/step
[*] 000362, d1[0.007] d2[0.015] g[107.672] ---> time[1.03], time_left[00:00:39]
1/1 [==============================] - 0s 36ms/step
[*] 000363, d1[0.005] d2[0.015] g[98.699] ---> time[0.83], time_left[00:00:30]
1/1 [==============================] - 0s 30ms/step
[*] 000364, d1[0.005] d2[0.015] g[98.596] ---> time[0.83], time_left[00:00:29]
1/1 [==============================] - 0s 23ms/step
[*] 000365, d1[0.006] d2[0.015] g[90.821] ---> time[0.70], time_left[00:00:24]
1/1 [==============================] - 0s 24ms/step
[*] 000366, d1[0.006] d2[0.015] g[94.058] ---> time[0.70], time_l

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000400.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000400.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000400.h5


In [61]:
import numpy as np
import cv2, os
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.models import load_model
from keras.preprocessing.image import img_to_array, save_img, array_to_img

#model_path = "<enter model path>"
# You can find the enhanced model on: /content/Enhancement Model/Model Output
model_path = "/content/Enhancement Model/Model Output/gen_model_000400.h5"
#image_path = "<enter image path>"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"
image_path = "/content/gans_new2/eval15/low/111.png"

# Processing Image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_arr = (img_to_array(img) - 127.5) / 127.5
resized = cv2.resize(img_arr, (256, 256), interpolation=cv2.INTER_AREA)
ready_img = np.expand_dims(resized, axis=0)

# Loading Model
model = load_model(model_path)

# Prdicting Image
pred = model.predict(ready_img)
pred = (cv2.medianBlur(pred[0], 1) + 1) / 2
pred = array_to_img(pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)
save_img("./output.png", pred)

1/1 [==============================] - 1s 1s/step


In [62]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def evaluate_model(model, image_path, target_path):
    # Load the low-light image
    low_light_img = cv2.imread(image_path)
    low_light_img = cv2.cvtColor(low_light_img, cv2.COLOR_BGR2RGB)
    low_light_img = (low_light_img.astype(np.float32) - 127.5) / 127.5
    low_light_img = cv2.resize(low_light_img, (256, 256))

    # Predict the enhanced image
    enhanced_img = model.predict(np.expand_dims(low_light_img, axis=0))
    enhanced_img = (enhanced_img[0] + 1) / 2  # Rescale the image to [0, 1]
    enhanced_img = cv2.resize(enhanced_img, (low_light_img.shape[1], low_light_img.shape[0]))

    # Convert the images to uint8 format
    low_light_img_uint8 = (low_light_img * 255).astype(np.uint8)
    enhanced_img_uint8 = (enhanced_img * 255).astype(np.uint8)

    # Calculate PSNR and SSIM
    psnr_score = psnr(low_light_img_uint8, enhanced_img_uint8)
    ssim_score = ssim(low_light_img_uint8, enhanced_img_uint8, multichannel=True)

    # Save the enhanced image
    cv2.imwrite(target_path, cv2.cvtColor(enhanced_img_uint8, cv2.COLOR_RGB2BGR))

    return psnr_score, ssim_score

# Example usage:
# Provide the path to your trained model and low-light image
# You can find the enhanced model on: /content/Enhancement Model/Model Output
model_path = "/content/Enhancement Model/Model Output/gen_model_000400.h5"
#Loading Test Data
#low_light_image_path = "/content/gans_new2/eval15/low/1.png"
low_light_image_path1 = "/content/gans_new2/eval15/low/111.png"
low_light_image_path2 = "/content/gans_new2/eval15/low/146.png"
low_light_image_path3 = "/content/gans_new2/eval15/low/179.png"
low_light_image_path4 = "/content/gans_new2/eval15/low/22.png"
low_light_image_path5 = "/content/gans_new2/eval15/low/1.png"
low_light_image_path6 = "/content/gans_new2/eval15/low/778.png"
low_light_image_path7 = "/content/gans_new2/eval15/low/665.png"
low_light_image_path8 = "/content/gans_new2/eval15/low/669.png"
low_light_image_path9 = "/content/gans_new2/eval15/low/748.png"
low_light_image_path10 = "/content/gans_new2/eval15/low/79.png"

#output_image_path = "./enhanced.png"
output_image_path1 = "./enhanced1.png"
output_image_path2 = "./enhanced2.png"
output_image_path3 = "./enhanced3.png"
output_image_path4 = "./enhanced4.png"
output_image_path5 = "./enhanced5.png"
output_image_path6 = "./enhanced6.png"
output_image_path7 = "./enhanced7.png"
output_image_path8 = "./enhanced8.png"
output_image_path9 = "./enhanced9.png"
output_image_path10 = "./enhanced10.png"

# Load the model
model = load_model(model_path)

# Evaluate the model and save the enhanced image
#psnr_score, ssim_score = evaluate_model(model, low_light_image_path, output_image_path)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path1, output_image_path1)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path2, output_image_path2)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path3, output_image_path3)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path4, output_image_path4)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path5, output_image_path5)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path6, output_image_path6)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path7, output_image_path7)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path8, output_image_path8)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path9, output_image_path9)
print("PSNR:", psnr_score)
print("SSIM:", ssim_score)
psnr_score, ssim_score = evaluate_model(model, low_light_image_path10, output_image_path10)

print("PSNR:", psnr_score)
print("SSIM:", ssim_score)


1/1 [==============================] - 0s 212ms/step
PSNR: 14.826695117881837
SSIM: 0.008717083629798759
1/1 [==============================] - 0s 18ms/step
PSNR: 13.162437865057123
SSIM: 0.01866931234775614
1/1 [==============================] - 0s 23ms/step


<ipython-input-62-975363da5111>:24: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score = ssim(low_light_img_uint8, enhanced_img_uint8, multichannel=True)


PSNR: 14.378689053900139
SSIM: 0.011911859268776926
1/1 [==============================] - 0s 24ms/step
PSNR: 14.50407949576441
SSIM: 0.03985955668487768
1/1 [==============================] - 0s 20ms/step
PSNR: 10.91589503499989
SSIM: 0.01805483230032998
1/1 [==============================] - 0s 21ms/step
PSNR: 21.176267378797956
SSIM: 0.12177248873829762
1/1 [==============================] - 0s 21ms/step
PSNR: 22.31082478761277
SSIM: 0.07779867042894108
1/1 [==============================] - 0s 39ms/step
PSNR: 15.952860007539762
SSIM: 0.015980976485428507
1/1 [==============================] - 0s 44ms/step
PSNR: 18.34124387167219
SSIM: 0.01767923026497205
1/1 [==============================] - 0s 36ms/step
PSNR: 14.108155307371597
SSIM: 0.004417250189536647
